<a href="https://colab.research.google.com/github/vladgap/Various/blob/ver1/MLN-091121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perceptron -- neuron with an activation function.
Inputs and their weights are numbered 0 to(n-1), bias is n

In [1]:
import numpy as np

# Single Neuron

In [2]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init)

    def set_activ(self, activ):
        """Overrides the 'linear' activation function."""
        self.activ = activ

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

In [3]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
# neu.set_activ('sigmoid')
neu.run([1,1])

5.0

## AND gate

In [4]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


In [5]:
#proverka
# a=[]
# a.append(Perceptron(inputs=2))
# a[1].weights

## OR gate

In [6]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [7]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [77]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: mln.network[layer][neuron].set_activ('linear').
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, eta = 0.5, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ):
        """Set the activation function to every neurons."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ) # set_activ for each neuron
    
    def set_output_activ(self, activ):
        """Set the activation function to the last (output) neurons."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = (y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def bp_regres(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the linear function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = (y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # STEP 3: Calculate the OUTPUT error terms
        self.d[-1] = (error) 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = fwd_error 
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def deriv(self, value, i, j=0):
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return value
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return value*(1-value)
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return value
          else:
            # print ('re<')
            return 0

    def bp_general(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative according each neuron's activation function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = (y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

## XOR gate=(OR+NAND)+AND

In [9]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10 -10  15] sigmoid
Layer 1 Neuron 1 : [ 15  15 -10] sigmoid
Layer 2 Neuron 0 : [ 10  10 -15] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [17]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(5000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

0.2639936036383276
0.2615704127958529
0.2605149789566715
0.258941641213415
0.25408777019167644
0.22570360041490783
0.07264303013666498
0.020142994318555388
0.010101782715039427
0.006463215084213302
0.004667390705052601
0.003618408509949823
0.0029379346186822353
0.002463915879304922
0.002116283556019731
0.0018512452444470545
0.0016429569269956624
0.0014752387721945217
0.0013374715661897934
0.001222411343397013
0.0011249551166589494
0.0010414090052690828
0.00096903610275965
0.0009057672072274925
0.0008500100635376779

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-5.63060033 -5.63537928  2.14149726] sigmoid
Layer 1 Neuron 1 : [ 4.83859788  4.83870974 -7.52474344] sigmoid
Layer 2 Neuron 0 : [-8.86495822 -8.77736649  4.31342937] sigmoid

XOR Gate:
0 0 = 0.0259749866
0 1 = 0.9704596276
1 0 = 0.9704264564
1 1 = 0.0278886477


# Example1 -- weights

In [10]:
print ("1 1 =",mln1.run([1,1]))
print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"

1 1 = [0.00715281]
[-10 -10  15]


In [11]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.87414972 -0.62185218  0.24012715] relu
Layer 1 Neuron 1 : [0.49433098 0.08444559 0.64471997] relu
Layer 2 Neuron 0 : [-0.83567208  0.29841863  0.50208051] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.87414972 -0.62185218  0.24012715] relu
Layer 1 Neuron 1 : [0.49433098 0.08444559 0.64471997] relu
Layer 2 Neuron 0 : [-0.83567208  0.29841863  0.50208051] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.87414972 -0.62185218  0.24012715] linear
Layer 1 Neuron 1 : [0.49433098 0.08444559 0.64471997] relu
Layer 2 Neuron 0 : [-0.83567208  0.29841863  0.50208051] sigmoid



# Example 2 -- sigmoid, bp_general, XOR

In [78]:
mln2 = MultiLayerNeuron(layers=[2,2,1], eta=0.5)
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(5000):
    MSE = 0.0
    MSE += mln2.bp_general([0,0],[0])
    MSE += mln2.bp_general([0,1],[1])
    MSE += mln2.bp_general([1,0],[1])
    MSE += mln2.bp_general([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

0.2901634770338539
0.24734675948506887
0.2022854950159863
0.16669289989994907
0.05820435342957007
0.01755739580333561
0.009098626046821408
0.005930143244535582
0.00433382916559958
0.003388023859978508
0.00276801400398334
0.002332611060113036
0.002011236451265689
0.0017649241997338997
0.0015704998379448885
0.0014133600028868145
0.0012838666364668875
0.001175413289489706
0.001083326371476366
0.0010042101685666556
0.0009355403034001427
0.0008754025387224505
0.0008223198792920612
0.0007751351009126971
0.0007329290844651672

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.45957362  4.46223168 -6.93940555] sigmoid
Layer 1 Neuron 1 : [-6.16366321 -6.18237906  2.38837026] sigmoid
Layer 2 Neuron 0 : [-9.19179012 -8.93444146  4.47518995] sigmoid

XOR Gate:
0 0 = 0.0237280121
0 1 = 0.9725196698
1 0 = 0.9724691805
1 1 = 0.0264875637


# Example 3 -- linear activs, bp_regres

In [14]:
mln3 = MultiLayerNeuron(layers=[3,1], eta=0.3)
# mln3.set_weights([[[4.9,-0.9,2.1,0.2]]])
# mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],[5.764286995])
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],[5.84646758])
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],[0.808633075])
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],[3.850298589])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Training Neural Network...

7.691871151091792
3.3555686591331836e-10

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973370e+00 -9.99625016e-01  1.99991681e+00  3.52286210e-05] linear



# Example 4 -- sigmoid, bp_regres

In [76]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] sigmoid
Layer 1 Neuron 1 : [ 1 -2  0  1] sigmoid
Layer 2 Neuron 0 : [-1  2  3] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 5.5167563003496065
MSE= 0.0027569211483228218
MSE= 0.0004342933976275543
MSE= 0.000170653351884855
MSE= 0.00013696142450800316

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1.45889838 -2.23585499  0.50338767  1.78022775] sigmoid
Layer 1 Neuron 1 : [ 0.91169201 -2.39503302  1.19558618  0.24752772] sigmoid
Layer 2 Neuron 0 : [0.8182108  1.56598495 2.1694304 ] linear



'predicted values:'

[3.5805677172677637,
 3.7245609127124464,
 3.749000704114528,
 3.120722177444701,
 3.8823249088088754,
 2.2582936588762523]

# Example 5 -- various, bp_general

In [82]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1,1])
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1], eta=0.3)
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_general([2,1,0],y4[0])
    MSE += mln4.bp_general([-1,0,1],y4[1])
    MSE += mln4.bp_general([0,0,0],y4[2])
    MSE += mln4.bp_general([1,1,0],y4[3])
    MSE += mln4.bp_general([-1,-1,-1],y4[4])
    MSE += mln4.bp_general([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] relu
Layer 1 Neuron 1 : [ 1 -2  0  1] relu
Layer 2 Neuron 0 : [1 0 1] sigmoid
Layer 3 Neuron 0 : [-1  2] linear



'original values:'

[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE= 0.02660867181761004
MSE= 0.000589264154624782
MSE= 0.0003192336234073572
MSE= 0.00026096488425357436
MSE= 0.00023326062943938882

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.02749394  0.59558381  0.8374333   0.71088629] relu
Layer 1 Neuron 1 : [-0.7916397   0.72739159 -0.79537478 -0.93308701] relu
Layer 2 Neuron 0 : [ 0.55525777 -0.68583543 -0.29534157] sigmoid
Layer 3 Neuron 0 : [0.70982803 0.72372912] linear



'predicted values:'

[1.0266103798576687,
 1.2608856965695243,
 1.0962649484926332,
 1.0537623324165222,
 1.0563508601036267,
 1.0065922328832158]